# Hosting models on Grid

## 2. Host model on a grid node

<h2>Import dependencies</h2>

In [1]:
import pickle
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import grid as gr
import torch.nn.functional as F
import torch.optim as optim
import syft as sy
from torch.utils.data import TensorDataset, DataLoader
import time

hook = sy.TorchHook(torch)

## Load model

In [2]:
import torchvision.models as models

def make_model(num_out_classes: int):
    """Load a resnet50 and add a new head to it."""
    model = models.resnet50(pretrained=True) 
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, num_out_classes)
    return model

model = make_model(7)

In [3]:
model.load_state_dict(torch.load("resnet-skin-cancer-detection"))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

<h2>Connect with remote workers</h2>
<strong><me>Before this step, it is necessary to initialize the workers separately<me></strong>

In [4]:
bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="bob")
bob.connect()

In [8]:
data = torch.zeros(torch.Size([1, 3, 224, 224]))
# traced_model = torch.jit.trace(model, data)
plan_model = sy.messaging.plan.method2plan(model.__call__)

In [9]:
plan_model(data)

TypeError: 'property' object is not callable

## Serve model

In [8]:
bob.models

['first_model']

In [9]:
bob.serve_model(traced_model, model_id="first_model")

'{"Error": "Model ID should be unique. There is already a model being hosted with this id."}'

In [10]:
bob.models

['first_model']

In [11]:
bob.run_inference(model_id="first_model", data=torch.zeros(1, 1, 28, 28))

{'prediction': [[-2.322812080383301,
   -2.303769111633301,
   -2.293975830078125,
   -2.3327362537384033,
   -2.3066728115081787,
   -2.274453639984131,
   -2.291421413421631,
   -2.3145298957824707,
   -2.3104403018951416,
   -2.2766435146331787]]}

W0819 13:09:54.048585 140515156686592 client.py:562] WebSocket connection was closed, aborting
W0819 13:10:12.241006 140515139901184 client.py:562] WebSocket connection was closed, aborting
